In [9]:
from collections import defaultdict
import numpy as np
import sys
from enum import Enum
class ACTION(Enum):
    high=1
    low=0 
    
class Winner:
    def __init__(self):
        self.P_H = 0.55
        self.P_L = 0.45
        self.state = 0
        self.done = 0
        self.R = 0
        self.highcost = -50
        self.lowcost = -10
        self.greedy = 0.1
        self.discount=0.9
        self.init_Q()
        self.alpha=0.1
    def reset_game(self):
        self.state=0
        self.done=0
    def ba_policy(self,state):
        smp = np.random.uniform(0,1)
        if smp >= self.greedy :
            if self.Q[state,ACTION.high] > self.Q[state,ACTION.low]:
                ac = ACTION.high
            else:
                ac = ACTION.low
        else:
            act = np.random.randint(0, 2)
            if act == 0:
                ac = ACTION.low
            else:
                ac = ACTION.high
        return ac
        
    #def ta_policy(self):
    def Qlearning(self,numepisode):
        self.init_Q()
        #self.init_C()
        for i_episode in range(1, numepisode+1):
            if i_episode % 1000 == 0:
                print("\rEpisode {}/{}.".format(i_episode, numepisode), end="")
                sys.stdout.flush()
            
            self.reset_game()
            state=0
            episode = []
            while(1):
                action = self.ba_policy(state)
                reward,next_state,done = self.env_step(state,action)
                if self.Q[next_state,ACTION.high] > self.Q[next_state,ACTION.low]:
                    next_atcion = ACTION.high
                else:
                    next_atcion = ACTION.low
                self.Q[state,action] = self.Q[state,action] + self.alpha*(reward + self.discount*self.Q[next_state,next_atcion]-self.Q[state,action])
                state = next_state
                if done == 1:
                    break

        #print(Q)
           
        return self.Q
        
        
    def env_step(self,state,action):
        done=0
        reward=0
        sample = np.random.uniform(0, 1)
        if action == ACTION.high:
            reward = reward + self.highcost
            suc_prob = self.P_H
        else:
            suc_prob = self.P_L
            reward = reward + self.lowcost
        if sample < suc_prob:
            state = state + 1
        else:
            state = state -1
        if state >= 3 :
            reward = reward + 1000
            done = 1
        if state <= (-3):
            done = 1
        return reward,state,done
    def init_Q(self):
        self.Q = defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                    self.Q[state-3,act]=0.0
    def init_C(self):
        
        self.C = defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                    self.C[state-3,act]=0.0
        

In [10]:
winner=Winner()

Q=winner.Qlearning(500000)
print('policy_value',Q)
import pandas as pd
buf = np.zeros((7,2))
for i in range(7):
    for j in ACTION:
        buf[i][j.value]=Q[i-3,j]

test_3 = pd.DataFrame(buf)
print(test_3)

Episode 500000/500000.policy_value defaultdict(<function Winner.init_Q.<locals>.<lambda> at 0x7f7b1c498400>, {(-3, <ACTION.high: 1>): 0.0, (-3, <ACTION.low: 0>): 0.0, (-2, <ACTION.high: 1>): -31.95584604893579, (-2, <ACTION.low: 0>): -1.1839296179820005, (-1, <ACTION.high: 1>): 4.535194522213999, (-1, <ACTION.low: 0>): 24.278776674289784, (0, <ACTION.high: 1>): 81.120436022826, (0, <ACTION.low: 0>): 63.80292811074652, (1, <ACTION.high: 1>): 138.97875263689065, (1, <ACTION.low: 0>): 249.08950677478128, (2, <ACTION.high: 1>): 571.9601162771231, (2, <ACTION.low: 0>): 461.69459027383425, (3, <ACTION.high: 1>): 0.0, (3, <ACTION.low: 0>): 0.0})
            0           1
0    0.000000    0.000000
1   -1.183930  -31.955846
2   24.278777    4.535195
3   63.802928   81.120436
4  249.089507  138.978753
5  461.694590  571.960116
6    0.000000    0.000000
